In [1]:
import numpy as np
import onnxruntime as rt
from PIL import Image

In [2]:
labels = ['hot_dog', 'not_hot_dog']

In [3]:
def transform_image_onnx(path, size): 
    image = Image.open(path)
    image = image.resize((size, size))
    # This returns a shape of (384, 384, 3) 
    image = np.array(image)
    # Convert it to (3, 384, 384)
    image = image.transpose(2, 0, 1).astype(np.float32)
    # Add batch dimension (1, 3, 384, 384)
    image = np.expand_dims(image, axis=0)
    # Convert values from 0-255 to 0-1 range
    image /= 255
    return image

In [4]:
hot_dog_tensor_onnx = transform_image_onnx('hot_dog_1.jpg', 384)
not_hot_dog_tensor_onnx = transform_image_onnx('not_hot_dog_1.jpg', 384)
hot_dog_tensor_onnx.shape

(1, 3, 384, 384)

In [5]:
sess = rt.InferenceSession('not-hot-dog.onnx')

In [6]:
sess.get_inputs()[0].name

'image_1_3_384_384'

In [7]:
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
input_dims = sess.get_inputs()[0].shape
output_dims = sess.get_outputs()[0].shape
input_name, input_dims, output_name, output_dims

('image_1_3_384_384', [1, 3, 384, 384], 'hot_dog', [1, 2])

In [8]:
results = sess.run([output_name], {input_name: hot_dog_tensor_onnx})[0]
labels[np.argmax(results)], results, labels

('not_hot_dog',
 array([[0.2517277 , 0.74827236]], dtype=float32),
 ['hot_dog', 'not_hot_dog'])

In [9]:
results = sess.run([output_name], {input_name: not_hot_dog_tensor_onnx})[0]
labels[np.argmax(results)], results, labels

('not_hot_dog',
 array([[0.38771978, 0.6122802 ]], dtype=float32),
 ['hot_dog', 'not_hot_dog'])